In [1]:
import pandas as pd
import os
# import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
path = './' + 'data/'
os.listdir(path)
# os.cpu_count()

['OD集货量',
 '中转场历史到件量0301至0531.csv',
 '中转场历史到件量0601至0831.csv',
 '场地数据.csv',
 '班次数据0601至0831.csv',
 '运力配载数据0601至0831']

In [2]:
%%time
# trans = pd.read_json('./data/运力配载数据0601至0831/line_info_0601.json', lines=True)  # 36.2 sec
hist_arr = pd.read_csv(path + '中转场历史到件量0601至0831.csv')
loc = pd.read_csv(path + '场地数据.csv')
# # # loc.to_csv(path + '场地数据.csv', index = None, encoding='utf_8_sig')
task_arrange = pd.read_csv(path + '班次数据0601至0831.csv')


path_od = path + 'OD集货量/sep/'   
dfs = pd.DataFrame()
for fname in os.listdir(path_od):
    if re.search(r'\.csv$', fname):
        dfs = pd.concat(
            [dfs , pd.read_csv(path_od + fname)], axis = 0, ignore_index = True)
prc_in = pd.DataFrame(dfs) 
print(
    'loc:', loc.shape, '\nhist:' ,
    hist_arr.shape, '\ntask:',
    task_arrange.shape, '\nod:' ,
    dfs.shape)
del dfs

### the story
以od为主表 通过时间和destination location join **history** 和 task ; location, 对task 和location中的缺失值进行处理.
location和task中的部分特征进行处理.
未来计划加入路线数据.

---

# Loc table clean  

In [2]:
loc = pd.read_pickle('./data_clean/loc.p'); loc.shape

(444, 21)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
loc[loc['场地代码'] == '552W']

In [26]:
# loc.columns
def loc_(loc):
    loc = loc.drop(['所在城市代码','应用场景', '分拨区' , '所在城市名称', '场地名称', '主服务城市','所在区部'], axis = 1) 
    loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
        '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']]  = loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
        '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']] == 0
    # loc[['装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']].describe()
    # loc.head(2)
    loc['off_1'] = loc['卸车卡位（17.5M）'] * 17.5
    loc['off_2'] = loc['卸车卡位（>=15M）'] * 15
    loc['off_3'] = loc['卸车卡位（<15M）'] * (13.5 * 0.970721 + 9.6 * 11.056306 + 7.6 * 2.279279 + 5.2 * 9.826577)/ \
        (0.970721 + 11.056306 + 2.279279 + 9.826577)
    loc['onn_1'] = loc['装车卡位（17.5M）'] * 17.5
    loc['onn_2'] = loc['装车卡位（15M）'] * 15
    loc['onn_3'] = loc['装车卡位（13.5M）'] * 13.5
    loc['onn_4'] = loc['装车卡位（9.6M）'] * 9.6
    loc['onn_5'] = loc['装车卡位（7.6M）'] * 7.6
    loc['onn_6'] = loc['装车卡位（5.2M）'] * 5.2
    loc['off'] = loc['off_1'] + loc['off_2'] + loc['off_3']
    loc['onn'] = loc['onn_1'] + loc['onn_2'] + loc['onn_3'] + loc['onn_4'] + loc['onn_5'] + loc['onn_6']
    loc = loc.drop(['off_1','off_2', 'off_3', 'onn_1', 'onn_2', 'onn_3', 'onn_4', 'onn_5', 'onn_6'], axis = 1)

    loc['off_p1'] = loc['off']/(loc['卸货卡位总数']+1)
    loc['off_p2'] = loc['off']/(loc['装货卡位总数']+1)
    loc['onn_p1'] = loc['onn']/(loc['卸货卡位总数']+1)
    loc['onn_p2'] = loc['onn']/(loc['装货卡位总数']+1)
    return loc




In [ ]:
# loc.to_pickle('./data_clean/loc.p')
# corr_loc = loc.corr()
# corr_loc.style.background_gradient(cmap='coolwarm', axis=None)

# Task arrage clean

In [ ]:
# task_arrange[(task_arrange['operate_zone_code'] == '552W') & (task_arrange['batch_date'] == '2021-06-04')]

In [ ]:
# pd.to_pickle(ta,  './data_clean/task.p')
ta = pd.read_pickle('./data_clean/task.p')
# task_loc[task_loc['operate_zone_code'] == '010R']

In [40]:
%%time
def ta_(ta):
    ta = ta.groupby(['operate_zone_code', 'batch_date']).\
        agg(min_begin = ('plan_begin_tm', min),
        max_begin = ('plan_begin_tm', max),
        min_end = ('plan_end_tm', min),
        max_end = ('plan_end_tm', max),
        count = ('batch_code', 'nunique')
        
        ).reset_index()

    def convt_min(col):
        ta[col] = ta[col].apply(lambda i: datetime.strptime(i,'%H:%M').hour*60 + datetime.strptime(i,'%H:%M').minute)
        
    for i in ['min_begin', 'max_begin', 'min_end','max_end']:
        convt_min(i)
    return ta


# ta['time_consume_per_vote'] = (ta['max_end'] - ta['min_begin'])/ta['count']

Wall time: 0 ns


In [ ]:
corr_ta = ta.corr()
corr_ta.style.background_gradient(cmap='coolwarm', axis=None)


In [ ]:
# %%time
# import matplotlib.pyplot as plt
# ta_plt2 = pd.DataFrame(ta['operate_zone_code'].value_counts())
# ta_plt2 = pd.DataFrame(ta_plt2['operate_zone_code'].value_counts())
# ta_plt2['site'] = ta_plt2.index
# plt.pie(x = 'operate_zone_code' , data =ta_plt2)

In [ ]:
# %%time
# task_loc = ta.merge(loc, left_on='operate_zone_code', right_on = '场地代码', how = 'left')

In [ ]:
%%time
del loc, ta 
# task_loc.to_pickle('./data_clean/task_loc.p')


In [ ]:
# %%time
# task_loc_valued = task_loc[~task_loc['场地代码'].isna()]
# task_loc_nulled = task_loc[task_loc['场地代码'].isna()]

In [ ]:
# task_loc.isnull().sum()/len(task_loc)

---
# od

In [ ]:
od_dest = pd.read_pickle('./data_clean/od_dest.p') 
"""
历史到件量的场地不是每个都需要考虑的吧 集货OD对应的场地应该只有400出头 场地数据基本能覆盖的
"""

'\n历史到件量的场地不是每个都需要考虑的吧 集货OD对应的场地应该只有400出头 场地数据基本能覆盖的\n'

In [ ]:
[prc_in[(prc_in['date1'] == '2021-06-08') & (prc_in['destsitecode'] == '552W')].sum(),
prc_in[(prc_in['date1'] == '2021-06-04') & (prc_in['srcsitecode'] == '552W')].sum()]

In [63]:
%%time
# prc_in.head()
def od_(od):
    od = od.groupby(['srcsitecode','destsitecode', 'date1']).agg(
        batch_cnt = ('srcbatchcode', 'nunique'),
        route_cnt = ('route_code', 'nunique'),
        votes = ('votes', sum),
        weights = ('weight1', sum),
        air_votes =('air_votes', sum),
        air_weights = ('air_weight1', sum)
    ).reset_index()

    od_dest = od.groupby(['destsitecode', 'date1']).agg(
        {'batch_cnt': sum, 
        'route_cnt': sum,
        'votes': sum, 
        'weights': sum,
        'air_votes': sum, 
        'air_weights':sum
        }).reset_index();od_dest
    return od_dest

Wall time: 0 ns


In [ ]:
# od.to_pickle('./data_clean/od.p')
# od_dest.to_pickle('./data_clean/od_dest.p')

In [ ]:
[od.date1.unique(), od.date1.max()]

---

# hist

In [ ]:
hist_arr[(hist_arr['a.zonecode'] == '552W') & ( hist_arr['a.report_dt'] == '2021-06-04' )]

In [ ]:
# hist_arr.to_pickle('./data_clean/hist_arr.p')
# merge all
hist_arr = pd.read_pickle('./data_clean/hist_arr.p')
hist_arr.head()

,a.report_dt,a.zonecode,all_arr_waybill,all_arr_weight,simu_arr_waybill,simu_arr_weight
0,2021-06-08,010CC006,525,1280.400000,0,0.000000
1,2021-06-08,010CC007,0,0.000000,0,0.000000
2,2021-06-08,010R,102944,230415.243664,76818,165515.803556
3,2021-06-08,010RA,109154,249089.557278,109154,249089.557278
4,2021-06-08,010RG,119183,261849.663955,118678,260439.168955


# data subset to 10 days


In [ ]:
# hist_arr_part = hist_arr[hist_arr['a.report_dt'] < '2021-06-11']

# od_dest = od.groupby(['destsitecode', 'date1']).agg(
#     {'batch_cnt': sum, 
#     'route_cnt': sum,
#     'votes': sum, 
#     'weights': sum,
#     'air_votes': sum, 
#     'air_weights':sum
#     }).reset_index();od_dest

--- 
# merge od_dest, ta, loc, hist_arr_part

In [ ]:
[ta.columns,
od_dest.columns,
loc.columns,
hist_arr.columns]

[Index(['operate_zone_code', 'batch_date', 'min_begin', 'max_begin', 'min_end',
        'max_end', 'count'],
       dtype='object'),
 Index(['destsitecode', 'date1', 'batch_cnt', 'route_cnt', 'votes', 'weights',
        'air_votes', 'air_weights'],
       dtype='object'),
 Index(['场地代码', '场地类型', '主服务业务区', '单小时处理能力', '卸货卡位总数', '卸车卡位（17.5M）',
        '卸车卡位（>=15M）', '卸车卡位（<15M）', '装货卡位总数', '装车卡位（17.5M）', '装车卡位（15M）',
        '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）', 'off', 'onn',
        'off_p1', 'off_p2', 'onn_p1', 'onn_p2'],
       dtype='object'),
 Index(['a.report_dt', 'a.zonecode', 'all_arr_waybill', 'all_arr_weight',
        'simu_arr_waybill', 'simu_arr_weight'],
       dtype='object')]

# 1126 测试集 限制场地为test set 当中的

In [ ]:
%%time
# os.listdir('testset')
test_out = pd.read_excel('./testset/结果输出格式&待预测场地列表.xlsx')
hist_test = pd.read_csv('./testset/中转场历史到件量day_1至day_6.csv')
loc_test = pd.read_excel('./testset/场地数据.xlsx')
od_test1 = pd.read_csv('./testset/od_data_day1_day5.csv')
od_test2 = pd.read_csv('./testset/od_data_day6_day10.csv')
ta_test =  pd.read_csv('./testset/班次day_1至day_10.csv')
# hist_test =  pd.read_csv('./testset/中转场历史到件量day_1至day_6.csv')
od_test = pd.concat([od_test1, od_test2], axis = 0)
# os.listdir('testset')


Wall time: 23.9 s


['od_data_day1_day5.csv',
 'od_data_day6_day10.csv',
 'OD集货量.7z.001',
 '中转场历史到件量day_1至day_6.csv',
 '场地数据.xlsx',
 '班次day_1至day_10.csv',
 '结果输出格式&待预测场地列表.xlsx']

In [42]:
%%time
ta_test = ta_(ta_test)
loc_test = loc_(loc_test)
loc_all = pd.concat([loc_test, loc], axis = 0).drop_duplicates()

In [65]:
%%time
od_test = od_(od_test)

Wall time: 16.3 s


In [67]:
od_test

,destsitecode,date1,batch_cnt,route_cnt,votes,weights,air_votes,air_weights
0,010R,day_10_weekday_7,49,50,49.0,138.470000,62.0,277.600000
1,010R,day_1_weekday_5,137,110,22.0,228.006667,3639.0,6778.520000
2,010R,day_2_weekday_6,20,20,20.0,30.330000,7.0,23.000000
3,010R,day_3_weekday_7,48,48,63.0,82.150000,77.0,223.300000
4,010R,day_4_weekday_1,168,145,66.0,174.100000,4093.0,7151.521364
...,...,...,...,...,...,...,...,...
4127,SIN01R,day_5_weekday_2,250,154,0.0,0.000000,860.0,3797.000000
4128,SIN01R,day_6_weekday_3,252,148,11.0,117.000000,885.0,4001.583333
4129,SIN01R,day_7_weekday_4,269,169,0.0,0.000000,867.0,4460.000000
4130,SIN01R,day_8_weekday_5,273,172,6.0,101.000000,993.0,5484.803030


In [66]:
ta_test['batch_date'].unique()

array(['day_10_weekday_7', 'day_1_weekday_5', 'day_2_weekday_6',
       'day_3_weekday_7', 'day_4_weekday_1', 'day_5_weekday_2',
       'day_6_weekday_3', 'day_7_weekday_4', 'day_8_weekday_5',
       'day_9_weekday_6'], dtype=object)

In [61]:
hist_arr['a.report_dt'].max()
# 日期不连续, 需要空两天.
# 训练集训练 直接预测两天, 再作补充. 再合并测试集训练, 预测剩余4天.

'2021-08-31'

In [ ]:
## 11246 don

In [ ]:
# od_dest.merge(ta, \
#     left_on=['destsitecode', 'date1'], 
#     right_on = ['operate_zone_code', 'batch_date'],
#     how = 'left',  validate = 'one_to_one')  .isna().sum()

In [72]:
def od2_(od_dest):
    od_dest['vper_ba'] = od_dest['batch_cnt'] / od_dest['votes'] 
    od_dest['vper_ro'] = od_dest['route_cnt'] / od_dest['votes'] 
    od_dest['vper_we'] = od_dest['weights']   / od_dest['votes'] 
    return od_dest

In [74]:
def merge_(od_dest, ta, hist_arr, od_ta_his):
    """
    od_dest , ta, loc, hist_arr_part
    """
    od_ta = od_dest.merge(ta, \
        left_on=['destsitecode', 'date1'], 
        right_on = ['operate_zone_code', 'batch_date'],
        how = 'left',  validate = 'one_to_one') 
        # task arrange lakc 267


    od_ta_his = hist_arr.merge(
        od_ta, 
        left_on = ['a.report_dt', 'a.zonecode'], 
        right_on = ['date1', 'destsitecode'], validate = 'one_to_one', how = 'right') \
            # .isna().sum()   264



    od_ta_his_loc = od_ta_his.merge(loc, \
        left_on=['destsitecode' ], 
        right_on = ['场地代码'],
        how = 'left')
        #  .isna().sum() 1388

    od_ta_his_loc = od_ta_his_loc.drop(['a.report_dt', 'a.zonecode','operate_zone_code','batch_date', \
        '场地代码','场地类型' ,'主服务业务区'], axis= 1)

In [76]:
%%time
od_ta_his_loc.isna().sum() 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
imp = SimpleImputer('median', missing_values = np.nan )
imp2 = KNNImputer(n_neighbors=12, weights="uniform", missing_values = np.nan)
# imp.fit_transform(od_ta_his[list(impute.index)])

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass missing_values=median as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [77]:
%%time
impute = pd.DataFrame(od_ta_his_loc.isna().sum() != 0)
impute = impute[impute[0] == True] 
for i in impute.index:
    od_ta_his_loc[i] = od_ta_his_loc[i].astype(float)

# # del loc, hist_arr  
 

Wall time: 190 ms


In [87]:
## 仅使用存在于测试集的站点.
od_ta_his_loc = od_ta_his_loc[od_ta_his_loc['destsitecode'].isin(list(test_out['zonecode']))]

In [ ]:
# od_ta_his_loc = od_ta_his_loc[['all_arr_waybill', 'all_arr_weight', 'simu_arr_waybill',
#        'simu_arr_weight', 'destsitecode', 'date1', 'batch_cnt', 'route_cnt',
#        'votes', 'weights', 'air_votes', 'air_weights']]

In [88]:
%%time
"""
knn imputation on missing values.
label encoding.
standard scale.
"""
od_ta_his_loc[impute.index.to_list()] = imp2.fit_transform(od_ta_his_loc[impute.index.to_list()])
# pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\pandas\core\frame.py:3673: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


NameError: name 'Pipeline' is not defined

In [90]:
# sns.color_palette("rocket", as_cmap=True)
# def plot_distribution(col):
#     f, axes = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
#     sns.boxplot(x = col,data = od_ta_his_loc, ax=axes[0], palette = "rocket")
#     sns.kdeplot(x = col,data = od_ta_his_loc, ax=axes[1], palette = "rocket") 

# plot_distribution('all_arr_waybill')

In [ ]:
# def lag_feature( df, lags, cols):
#     for col in cols:
#         print(col)
#         tmp = df[['destsitecode',col ]]
#         for i in lags:
#             shifted = tmp.copy()
#             shifted.columns = ['destsitecode', col + "_lag_"+str(i)]
#             shifted.date_block_num = shifted.date_block_num + i
#             df = pd.merge(df, shifted, on=['destsitecode'], how='left')
#     return df

In [ ]:
# hist_od_part = hist_arr_part.merge(
#     od_dest, 
#     left_on = ['a.report_dt', 'a.zonecode'], 
#     right_on = ['date1', 'destsitecode'], validate = 'one_to_one', how = 'right'); len(hist_od_part)

# hist_od_part = hist_od_part.drop(['a.report_dt','a.zonecode'], axis =1 )

In [ ]:
# del od, task_loc, hist_arr
# del task_loc_nulled, task_loc_valued
# # [len(task_loc), len(train), len(od)]

---
# what a model

In [92]:
# od_ta_his_loc.columns

In [245]:
od_ta_his_loc['destsitecode'] = od_ta_his_loc['destsitecode'].astype(str)
train_data = od_ta_his_loc
# [od_ta_his_loc['destsitecode'] == '010W'] 
train_x = train_data.drop(['all_arr_weight', 'simu_arr_waybill','simu_arr_weight'], axis = 1)
# train_x = train_x[['all_arr_waybill', 'date1', 'batch_cnt', 'route_cnt',
#        'votes', 'weights', 'air_votes', 'air_weights']]
train_x.index = pd.to_datetime(train_x['date1'])
train_x = train_x.rename({'all_arr_waybill' : 'y' }, axis = 1)

# train_x = train_x.asfreq('D')
# train_x = train_x.rename({'all_arr_waybill' : 'y' }, axis = 1)
cols = [i for i in train_x.columns if i not in ['y', 'date1', 'destsitecode']]

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_33900/4012905181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  od_ta_his_loc['destsitecode'] = od_ta_his_loc['destsitecode'].astype(str)


In [255]:
# from sktime.forecasting.model_selection import temporal_train_test_split
# from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [257]:
list = []
final = pd.DataFrame()
df_naiv = train_x.rename({'date1' : 'ds'}, axis = 1)
# df_naiv_test = x_test.rename({'date1' : 'ds'}, axis = 1)
site = list(df_naiv['destsitecode'].unique())
for g in tqdm(site):
   print(str(g))
   group = df_naiv[df_naiv['destsitecode'] == g]
   # x_train, x_test = temporal_train_test_split(group, train_size = .75)
   del m
   m = Prophet()
   [m.add_regressor(i) for i in cols]
   m.fit(group)
   test = x_test[x_test['destsitecode'] == x_train['destsitecode'][0]]
   print(test.shape)
   print(m.predict(test)['yhat'])
   list.extend(m.predict(test)['yhat'])

   
# future = m.make_future_dataframe(periods = 365)
# forecast = m.predict(future)
# forecast = forecast.rename(columns = {
#    'yhat': 'yhat_' + g
# })
# final = pd.merge(final, forecast.set_index('ds'), how = 'outer', left_index = True, right_index = True)

# final = final[['yhat_' + g
#    for g in grouped.groups.keys()
# ]]

  0%|          | 0/310 [00:00<?, ?it/s]

010R


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 22.


(10, 35)
0     24571.844017
1     63173.078524
2     55872.502199
3     -2596.925231
4     76151.176263
5    110529.661711
6     17680.180854
7     55480.475989
8    612824.860112
9    777577.242122
Name: yhat, dtype: float64


  0%|          | 1/310 [00:21<1:52:29, 21.84s/it]

010RG


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 13.
  0%|          | 1/310 [00:22<1:57:28, 22.81s/it]


RuntimeError: Initialization failed.

In [179]:
# for i in grouped.groups:
#    print(i)
# grouped.get_group('010R')

In [170]:
%%time
pred = m.predict(valid_group)['yhat']

Wall time: 15 s


In [188]:
df_naiv_test[['y']]

,y,destsitecode,ds,batch_cnt,route_cnt,votes,weights,air_votes,air_weights,vper_ba,...,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）,off,onn,off_p1,off_p2,onn_p1,onn_p2
date1,,,,,,,,,,,,,,,,,,,,,
2021-06-17,86787.0,716VA,2021-06-17,1217,1275,55702.0,163122.249297,4086.0,10991.250476,0.021848,...,1.0,0.0,1.0,1.0,25.27636,58.8,3.610909,1.685091,8.400000,3.920
2021-06-18,93603.0,716VA,2021-06-18,1228,1295,58354.0,167541.586897,3990.0,10494.142308,0.021044,...,1.0,0.0,1.0,1.0,25.27636,58.8,3.610909,1.685091,8.400000,3.920
2021-06-19,86880.0,716VA,2021-06-19,1179,1256,51264.0,142605.290238,3131.0,9360.866667,0.022999,...,1.0,0.0,1.0,1.0,25.27636,58.8,3.610909,1.685091,8.400000,3.920
2021-06-20,74955.0,716VA,2021-06-20,1106,1188,41161.0,117409.602674,2402.0,7088.850000,0.026870,...,1.0,0.0,1.0,1.0,25.27636,58.8,3.610909,1.685091,8.400000,3.920
2021-06-21,78191.0,716VA,2021-06-21,1156,1237,47396.0,126828.305555,3562.0,9047.105758,0.024390,...,1.0,0.0,1.0,1.0,25.27636,58.8,3.610909,1.685091,8.400000,3.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-27,170394.0,991W,2021-08-27,1132,1261,89385.0,197033.750819,17902.0,39490.549433,0.012664,...,1.0,1.0,1.0,1.0,22.77636,53.4,2.070578,0.911054,4.854545,2.136
2021-08-28,187065.0,991W,2021-08-28,1024,1180,76602.0,160450.382720,12356.0,26192.396667,0.013368,...,1.0,1.0,1.0,1.0,22.77636,53.4,2.070578,0.911054,4.854545,2.136
2021-08-29,157627.0,991W,2021-08-29,1005,1151,71607.0,142345.344013,10496.0,22829.670000,0.014035,...,1.0,1.0,1.0,1.0,22.77636,53.4,2.070578,0.911054,4.854545,2.136


In [189]:
# %%time

# from fbprophet import Prophet
# # df = x[['CREATION_DATE_TIME_STAMP', 'qty']]
# df_naiv = x_train 
# df_naiv = df_naiv.rename({'date1' : 'ds'}, axis = 1)
# df_naiv_test = x_test
# df_naiv_test = df_naiv_test.rename({'date1' : 'ds'}, axis = 1)
# m = Prophet()
# [m.add_regressor(i) for i in cols]

# m.fit(df_naiv)

# # from sklearn.metrics import mean_absolute_percentage_error

# # # m.predict(x_test)
# # mean_absolute_percentage_error(df_naiv_test['y'], m.predict(df_naiv_test)['yhat'])
# # ss

In [ ]:
# m.plot_components(m.predict(df_naiv))

In [ ]:
# # %%warning = False
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF

# test_pre_p = go.Figure()
# test_pre_p.add_trace(go.Scatter(x = test_pred['ds'],\
#     y = test_pred['yhat'], 
#     mode = 'markers',
#     name = 'pred'))
# test_pre_p.add_trace(go.Scatter(x = test_all['ds'], \
#     y = test_all['y'], 
#     mode = 'markers',
#     name = 'original_out'))


In [ ]:
# !jupyter nbconvert data_clea.ipynb --to html --no-input --no-prompt --stdout

# darts

In [121]:
train_x = train_x.reset_index(drop = True)
train_x['date1'] = pd.to_datetime(train_x['date1'])
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_percentage_error
x_train, x_test = temporal_train_test_split(train_x, train_size = .7)

In [131]:
# darts
from darts import TimeSeries
cols = list(x_train.columns[2:])
series_train = TimeSeries.from_dataframe(x_train, 
       time_col = 'date1', 
       value_cols ='all_arr_waybill')
series_test = TimeSeries.from_dataframe(x_test, 
       time_col = 'date1', 
       value_cols =  'all_arr_waybill')
series_train_cov = TimeSeries.from_dataframe(x_train,\
       'ds',
       cols)
series_test_cov = TimeSeries.from_dataframe(x_test, 
       'ds', 
       cols)

[2021-11-27 19:58:08,871] ERROR | darts.timeseries | ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency cannot be inferred.


ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency cannot be inferred.

In [ ]:
ful_span = TimeSeries.from_dataframe(x_train.append(x_test), \
    'ds', 
    value_cols = 'y')
ful_span_cov = TimeSeries.from_dataframe(x_train.append(x_test), \
    'ds', 
    cols )

In [ ]:
%%time
LSTM_darts = RNNModel(
    model='LSTM',
    hidden_dim=20,
    dropout=0,
    batch_size=7,
    n_epochs=5,
    optimizer_kwargs={'lr': 1e-3},
    model_name='lstm_RNN',
    log_tensorboard=True,
    random_state=42,
    # training_length=20,
    input_chunk_length=31,
    force_reset=True
)
LSTM_darts.fit(series_train,
             future_covariates=series_train_cov,
            #  val_series=series_test,
            #  val_future_covariates=series_test_cov,
             verbose=True)

In [ ]:
np.set_printoptions(suppress=True)
lstm_pred_test = LSTM_darts.predict(n = 334, future_covariates = ful_span_cov).values()